In [1]:
# import relevant libraries

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# load all csv files with headers, except ftp-write, which does not have column names
df_back = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back.csv")
df_buffer_overflow = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_BufferOverflow.csv")
df_guess_password = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_GuessPassword.csv")
df_neptune = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Neptune.csv")
df_nmap = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_NMap.csv")
df_normal = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Normal.csv")
df_port_sweep = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_PortSweep.csv")
df_rootkit = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_RootKit.csv")
df_satan = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Satan.csv")
df_smurf = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Smurf.csv")

In [3]:
# load ftp-write without headers
df_ftp_write = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_FTPWrite.csv", header = None)

In [4]:
# verify import

df_back.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.001,0.001,0.1,0,0.100,0,0.0,0.0,0.0,0.0
1,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.002,0.002,0.1,0,0.050,0,0.0,0.0,0.0,0.0
2,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.003,0.003,0.1,0,0.033,0,0.0,0.0,0.0,0.0
3,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.004,0.004,0.1,0,0.025,0,0.0,0.0,0.0,0.0
4,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.005,0.005,0.1,0,0.020,0,0.0,0.0,0.0,0.0


In [5]:
# assign correct column names to df_ftp_write
df_ftp_write.columns = df_back.columns

In [6]:
# verify 
df_ftp_write.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0026,0,0.07,0,0.00116,0.00451,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0
1,0.0134,0,0.34,0,0.00100,0.39445,0,0,0.2,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
2,0.0000,0,0.14,0,0.00613,0.00000,0,0,0.0,0.0,...,0.001,0.084,0.10,0.0,0.10,0.002,0,0,0,0
3,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.085,0.10,0.0,0.10,0.002,0,0,0,0
4,0.0032,0,0.07,0,0.00104,0.00449,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0


In [7]:
# check dataframe info for basic EDA
dfs = [df_back, df_buffer_overflow, df_ftp_write, df_guess_password, 
                      df_neptune, df_nmap, df_normal, df_port_sweep, df_rootkit, 
                      df_satan, df_smurf]

for df in dfs:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   duration                      968 non-null    float64
 1    protocol_type                968 non-null    int64  
 2    service                      968 non-null    int64  
 3    flag                         968 non-null    float64
 4    src_bytes                    968 non-null    float64
 5    dst_bytes                    968 non-null    float64
 6    land                         968 non-null    int64  
 7    wrong_fragment               968 non-null    int64  
 8    urgent                       968 non-null    int64  
 9    hot                          968 non-null    float64
 10   num_failed_logins            968 non-null    int64  
 11   logged_in                    968 non-null    float64
 12   num_compromised              968 non-null    float64
 13   root

**No nulls are present in any dataframe. Proceed with appending the dataframes.**

In [8]:
# add an attack column in each df, and label the rows with the respective attack type

df_back['attack'] = 'back'
df_buffer_overflow['attack'] = 'buffer overflow'
df_ftp_write['attack'] = 'ftpwrite'
df_guess_password['attack'] = 'guess password'
df_neptune['attack'] = 'neptune'
df_nmap['attack'] = 'nmap'
df_normal['attack'] = 'normal'
df_port_sweep['attack'] = 'port sweep'
df_rootkit['attack'] = 'rootkit'
df_satan['attack'] = 'satan'
df_smurf['attack'] = 'smurf'

In [9]:
# verify

df_smurf.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.001,0.001,0.002,0.001,0,0.0,0,0.0,0,smurf
1,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.002,0.001,0.002,0.001,0,0.0,0,0.0,0,smurf
2,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.003,0.002,0.002,0.002,0,0.0,0,0.0,0,smurf
3,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.004,0.003,0.002,0.003,0,0.0,0,0.0,0,smurf
4,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.005,0.003,0.002,0.003,0,0.0,0,0.0,0,smurf


In [10]:
# append the dataframes into one final_df

final_df = pd.concat([df_back, df_buffer_overflow, df_ftp_write, df_guess_password, 
                      df_neptune, df_nmap, df_normal, df_port_sweep, df_rootkit, 
                      df_satan, df_smurf], ignore_index=True)

In [15]:
# verify append

final_df.tail(15)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
817536,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.241,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817537,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.242,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817538,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.243,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817539,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.244,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817540,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.245,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817541,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.246,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817542,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.247,0.097,0.001,0.097,0.0,0.0,0.0,0.0,0.0,smurf
817543,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.248,0.097,0.001,0.097,0.0,0.0,0.0,0.0,0.0,smurf
817544,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.249,0.098,0.001,0.098,0.0,0.0,0.0,0.0,0.0,smurf
817545,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.250,0.098,0.001,0.098,0.0,0.0,0.0,0.0,0.0,smurf


## Binary Classification

In [16]:
binary_clf_df = final_df.copy()

# make attack_binary column - any attack type == 1, normal == 0
binary_clf_df['attack_binary'] = binary_clf_df['attack'].apply(lambda x: 0 if x =='normal' else 1)

In [42]:
# verify column creation

binary_clf_df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,attack_binary
0,0.0,0.0,0.0,0.0,0.5454,0.08314,0,0.0,0.0,0.2,...,0.1,0.0,0.100,0.0,0.0,0.0,0.0,0.0,back,1
1,0.0,0.0,0.0,0.0,0.5454,0.08314,0,0.0,0.0,0.2,...,0.1,0.0,0.050,0.0,0.0,0.0,0.0,0.0,back,1
2,0.0,0.0,0.0,0.0,0.5454,0.08314,0,0.0,0.0,0.2,...,0.1,0.0,0.033,0.0,0.0,0.0,0.0,0.0,back,1
3,0.0,0.0,0.0,0.0,0.5454,0.08314,0,0.0,0.0,0.2,...,0.1,0.0,0.025,0.0,0.0,0.0,0.0,0.0,back,1
4,0.0,0.0,0.0,0.0,0.5454,0.08314,0,0.0,0.0,0.2,...,0.1,0.0,0.020,0.0,0.0,0.0,0.0,0.0,back,1


In [17]:
# define X and target

X = binary_clf_df.drop(columns=['attack', 'attack_binary'])
y = binary_clf_df['attack_binary']

In [18]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)

In [20]:
# resample training data

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [25]:
# check distribution
print(f'Original dataset shape: {y_train.value_counts()}')
print(f"Resampled dataset shape: {y_train_resampled.value_counts()}")

Original dataset shape: attack_binary
0    461567
1    192473
Name: count, dtype: int64
Resampled dataset shape: attack_binary
0    461567
1    461567
Name: count, dtype: int64


In [26]:
# initialize the XGBoost Classifier

xgb_clf = XGBClassifier(
    objective='binary:logistic',  # For binary classification
    eval_metric='logloss',        # Loss function for binary classification
    use_label_encoder=False       # Required for recent versions of XGBoost
)

# train the model on the SMOTE-resampled data
xgb_clf.fit(X_train_resampled, y_train_resampled)

/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:43:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [28]:
# perform 5-fold cross-validation for overfitting check
scores = cross_val_score(xgb_clf, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {scores.mean()}")

/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:44:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:44:26] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:44:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:44:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW

Cross-validation accuracy: 0.9999274211392033


In [29]:
# make predictions on test set 

y_pred = xgb_clf.predict(X_test)

In [30]:
# evaluate model performance

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9999143788491294
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    115143
           1       1.00      1.00      1.00     48368

    accuracy                           1.00    163511
   macro avg       1.00      1.00      1.00    163511
weighted avg       1.00      1.00      1.00    163511



In [31]:
# check test vs train accuracy to check for overfitting

# accuracy on the training set
y_train_pred = xgb_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training accuracy: {train_accuracy}")

# accuracy on the test set
y_test_pred = xgb_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test accuracy: {test_accuracy}")

Training accuracy: 0.9999984710415265
Test accuracy: 0.9999143788491294


In [39]:
# select a single row from the test set to test the model

test_index = 65  # change the index to test different rows
single_row = X_test.iloc[[test_index]]  # Select the test row (as a DataFrame)
actual_label = y_test.iloc[test_index]  # Get the actual label for that row

# make a prediction for the single row
predicted_label = xgb_clf.predict(single_row)
print(f"Test Row {test_index} - Actual Label: {actual_label}, Predicted Label: {predicted_label[0]}")

# evaluate whether the prediction matches the actual label
if actual_label == predicted_label[0]:
    print(f"The model classified the row correctly!")
else:
    print(f"The model misclassified the row.")

Test Row 65 - Actual Label: 1, Predicted Label: 1
The model classified the row correctly!


## Multiclassification with XGBoost

In [43]:
multi_clf_df = final_df.copy()

In [44]:
# label encode the attack column

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(multi_clf_df['attack'])  # convert attack types into numeric values

In [45]:
# define X and y variables

X = final_df.drop(columns = ['attack'])

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# apply SMOTE
smote = SMOTE(random_state = 42)

# resample on train
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [47]:
# initialize the XGBoost classifier for multiclass classification
xgb_clf = XGBClassifier(
    objective='multi:softmax',  
    num_class=len(np.unique(y)),
    eval_metric='mlogloss',    
    use_label_encoder=False    
)

# train the model on the SMOTE-resampled data
xgb_clf.fit(X_resampled, y_resampled)

/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:01:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=11, num_parallel_tree=None, ...)

In [48]:
# make predictions on the test data
y_pred = xgb_clf.predict(X_test)

# decode the predictions back into the original attack type labels
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# evaluate the model performance
print(f"Accuracy: {accuracy_score(y_test_labels, y_pred_labels)}")
print(classification_report(y_test_labels, y_pred_labels))

Accuracy: 0.999926610442111


/Users/adityamxr/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                 precision    recall  f1-score   support

           back       0.99      1.00      1.00       192
buffer overflow       1.00      0.88      0.93         8
       ftpwrite       0.50      1.00      0.67         1
 guess password       1.00      1.00      1.00         9
        neptune       1.00      1.00      1.00     45575
           nmap       1.00      1.00      1.00       301
         normal       1.00      1.00      1.00    115143
     port sweep       0.99      1.00      1.00       582
        rootkit       0.00      0.00      0.00         3
          satan       1.00      1.00      1.00      1098
          smurf       1.00      1.00      1.00       599

       accuracy                           1.00    163511
      macro avg       0.86      0.90      0.87    163511
   weighted avg       1.00      1.00      1.00    163511



/Users/adityamxr/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [49]:
# Accuracy on the training set
y_train_pred = xgb_clf.predict(X_resampled)
train_accuracy = accuracy_score(y_resampled, y_train_pred)
print(f"Training accuracy: {train_accuracy}")

# Accuracy on the test set
y_test_pred = xgb_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test accuracy: {test_accuracy}")

Training accuracy: 0.9999998030424816
Test accuracy: 0.999926610442111


In [50]:
# perform 5-fold cross-validation on the SMOTE-resampled data
cross_val_scores = cross_val_score(xgb_clf, X_resampled, y_resampled, cv=5, scoring='accuracy')

# print CV scores
print(f"Cross-validation accuracy: {cross_val_scores.mean()} (std: {cross_val_scores.std()})")

/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:03:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:04:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:05:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/adityamxr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [19:06:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserW

Cross-validation accuracy: 0.9999917277846526 (std: 3.869601137945189e-06)


In [51]:
single_test_row = X_test.iloc[58].values.reshape(1, -1)

# model makes a prediction on the single row
predicted_class_numeric = xgb_clf.predict(single_test_row)

# decode the numeric prediction back into the original attack type
predicted_class_label = label_encoder.inverse_transform(predicted_class_numeric)

print(f"Predicted Attack Type: {predicted_class_label[0]}")

# compare it with the actual label for that row
actual_class_numeric = y_test[58]
actual_class_label = label_encoder.inverse_transform([actual_class_numeric])

print(f"Actual Attack Type: {actual_class_label[0]}")

Predicted Attack Type: neptune
Actual Attack Type: neptune


## Simple GUI for Testing Model Predictions

In [54]:
import tkinter as tk
from tkinter import ttk

# function to predict and display results
def predict_and_display():
    try:
        selected_index = int(index_entry.get()) 
        
        if 0 <= selected_index < len(X_test):  # ensure the index is valid
            single_row = X_test.iloc[[selected_index]]  
            actual_label = y_test[selected_index]  
            
            # predict the label
            predicted_label = xgb_clf.predict(single_row)[0]
            
            # decode labels (Ensure label_encoder is fitted and defined)
            actual_attack = label_encoder.inverse_transform([actual_label])[0]
            predicted_attack = label_encoder.inverse_transform([predicted_label])[0]
            
            # format the result string
            result_text = f"Test Row {selected_index}:\n"
            result_text += f"Actual Attack: {actual_attack}\n"
            result_text += f"Predicted Attack: {predicted_attack}\n"
            
            if actual_label == predicted_label:
                result_text += "\nThe model classified the row correctly!"
            else:
                result_text += "\nThe model misclassified the row."
            
            result_label.config(text=result_text)
        else:
            result_label.config(text="Invalid index. Please enter a valid row number.")
    except ValueError:
        result_label.config(text="Please enter a valid integer index.")

# create the main window
root = tk.Tk()
root.title("Network Intrusion Prediction")

# create a frame for padding and better layout
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# row index input label and entry field
ttk.Label(frame, text="Enter row index from X_test:").grid(column=0, row=0, sticky=tk.W)
index_entry = ttk.Entry(frame, width=10)
index_entry.grid(column=1, row=0, sticky=(tk.W, tk.E))

# predict button
ttk.Button(frame, text="Predict", command=predict_and_display).grid(column=2, row=0, sticky=tk.W)

# label to display the result
result_label = ttk.Label(frame, text="")
result_label.grid(column=0, row=1, columnspan=3, sticky=(tk.W, tk.E))

# start the Tkinter GUI event loop
root.mainloop()